In [4]:
# original size 258866 # 258841 # 258816
import sys
import os
from torch.utils.data import DataLoader
sys.path.append("..")
from tqdm import tqdm
import numpy as np
import pandas as pd

from utils import train_test_split, create_metadata_df
from dataset_xy import CARLADatasetXY
import json
from send2trash import send2trash
import shutil

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Move sensor folders that we are not using

In [42]:
root_dir = os.path.join("..", "..", "data", "data")
keep_input = ["lidar", "rgb", "measurements"] # "lidar"

In [53]:
def move_unused_sensors_to_new_folder(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                path_parts = root.split(os.sep)
                idx_data_first = path_parts.index("data")
                path_parts[idx_data_first + 1] += " unused"
                dir_new = os.path.join(*path_parts)
                if not os.path.exists(dir_new):
                    os.makedirs(dir_new)
                shutil.move(root, dir_new)

# Move folders back into the data folder

In [119]:
root_dir = os.path.join("..", "..", "data", "data unused")
#root_dir = "/Users/julianvonklitzing/Documents/GitHub/end2endappras/data/data unused"

In [120]:
for (root, dirs, files) in os.walk(root_dir, topdown=True):
    # Current folder contains the files
    if not dirs:
        dir, input_type = os.path.split(root)
        dir_parts = dir.split("/")
        idx = dir_parts.index("data") # idx of first data
        dir_parts[idx + 1] = "data"
        dir_new = os.path.join(*dir_parts)
        shutil.move(root, dir_new)

# Move/ Delete _prep folders

In [ ]:
def move_unused_sensors_to_new_folder(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                path_parts = root.split(os.sep)
                idx_data_first = path_parts.index("data")
                path_parts[idx_data_first + 1] += " unused"
                dir_new = os.path.join(*path_parts)
                if not os.path.exists(dir_new):
                    os.makedirs(dir_new)
                shutil.move(root, dir_new)

In [3]:
def delete_sensor_folders(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                shutil.rmtree(root)

In [4]:
# root_dir = os.path.join("..", "..", "data", "data")
# keep_input = ["lidar", "lidar_prep", "rgb", "measurements"] # "lidar"

# delete_sensor_folders(root_dir, keep_input)

# Delete entries with NaN values

In [101]:
path_data = os.path.join("..", "..", "data", "data")

config_xy = {"used_inputs": ["measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])

# Create Dataset & DataLoader
dataset = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data, config=config_xy)
dataloader = DataLoader(dataset=dataset, batch_size=64, num_workers=0, sampler=None, shuffle=False)

In [117]:
path_data = os.path.join("..", "..", "data", "data unused")

config_xy = {"used_inputs": ["lidar"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# Create df_meta 
df_meta_data_lidar = create_metadata_df(path_data, config_xy["used_inputs"])

# Create Dataset & DataLoader
# dataset = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data, config=config_xy)
# dataloader = DataLoader(dataset=dataset, batch_size=64, num_workers=0, sampler=None, shuffle=False)

In [118]:
df_meta_data_lidar

,dir,lidar
0,../../data/data unused/cycl_dataset_23_11/Rout...,0000.npy
1,../../data/data unused/cycl_dataset_23_11/Rout...,0001.npy
2,../../data/data unused/cycl_dataset_23_11/Rout...,0002.npy
3,../../data/data unused/cycl_dataset_23_11/Rout...,0003.npy
4,../../data/data unused/cycl_dataset_23_11/Rout...,0004.npy
...,...,...
258811,../../data/data unused/dirt_dataset_23_11/Rout...,0189.npy
258812,../../data/data unused/dirt_dataset_23_11/Rout...,0190.npy
258813,../../data/data unused/dirt_dataset_23_11/Rout...,0191.npy
258814,../../data/data unused/dirt_dataset_23_11/Rout...,0192.npy


In [103]:
df_meta_data["num"] = df_meta_data["measurements"].apply(lambda x: x.split(".")[0])
df_meta_data["dir"] = df_meta_data["dir"].apply(lambda x: os.path.join(*x.split(os.sep)[4:]))

df_meta_data_lidar["num"] = df_meta_data_lidar["lidar"].apply(lambda x: x.split(".")[0])
df_meta_data_lidar["dir"] = df_meta_data_lidar["dir"].apply(lambda x: os.path.join(*x.split(os.sep)[4:]))
df_meta_data_lidar = df_meta_data_lidar.reset_index()

In [104]:
df_merge = df_meta_data_lidar.merge(right=df_meta_data, on=["dir", "num"], how="outer",indicator=True)

In [105]:
df_merge

,index,dir,lidar,num,measurements,_merge
0,0,cycl_dataset_23_11/Routes_non-straight-junctio...,0000.npy,0000,0000.json,both
1,1,cycl_dataset_23_11/Routes_non-straight-junctio...,0001.npy,0001,0001.json,both
2,2,cycl_dataset_23_11/Routes_non-straight-junctio...,0002.npy,0002,0002.json,both
3,3,cycl_dataset_23_11/Routes_non-straight-junctio...,0003.npy,0003,0003.json,both
4,4,cycl_dataset_23_11/Routes_non-straight-junctio...,0004.npy,0004,0004.json,both
...,...,...,...,...,...,...
258861,258861,dirt_dataset_23_11/Routes_Scenario1_Town01_cur...,0189.npy,0189,0189.json,both
258862,258862,dirt_dataset_23_11/Routes_Scenario1_Town01_cur...,0190.npy,0190,0190.json,both
258863,258863,dirt_dataset_23_11/Routes_Scenario1_Town01_cur...,0191.npy,0191,0191.json,both
258864,258864,dirt_dataset_23_11/Routes_Scenario1_Town01_cur...,0192.npy,0192,0192.json,both


In [109]:
idx_lidar_to_delete = df_merge[df_merge["_merge"] == "left_only"].index.tolist()

In [59]:
def measurements_to_df(dataloader):
    idxs, speed, steer, throttle, brake, command = [], [], [], [], [], []
    for idx, (x, y) in tqdm(enumerate(dataloader)):
        idxs.append(idx)
        speed += x['speed'].flatten().tolist()
        command += x['command'].flatten().tolist()
        steer += y['steer'].flatten().tolist()
        throttle += y['throttle'].flatten().tolist()
        brake += y['brake'].flatten().tolist()

    df_measurements = pd.DataFrame({"speed": speed, "command": command, "steer": steer, "throttle": throttle, "brake": brake}, index=list(range(len(speed))))
    df_measurements.to_pickle("measurements_.pickle")
    return df_measurements

def remove_entries_at_indices(idx_nan_list, df_meta_entire):
    count_files_deleted = 0
    for idx_nan in idx_nan_list:
        path_nan = os.path.join(df_meta_entire.loc[idx_nan]["dir"], "measurements", df_meta_entire.loc[idx_nan]["measurements"])
        with open(path_nan, 'r') as f:
            measurements = json.load(f)
        if not np.isnan(measurements["steer"]):
            print("Break: indexes do not fit!")
            break
        dir = df_meta_entire["dir"].loc[idx_nan]
        dirs_sensors = os.listdir(dir)
        number_entry = df_meta_entire["measurements"].loc[idx_nan].split(".")[0]
        for dir_sensor in dirs_sensors:
            if not dir_sensor.startswith("."):
                files = os.listdir(os.path.join(dir, dir_sensor))
                for file in files:
                    if file.startswith(number_entry):
                        count_files_deleted += 1
                        # os.remove(os.path.join(dir, dir_sensor, file))
                        send2trash(os.path.join(dir, dir_sensor, file))
    return count_files_deleted

def remove_steer_nan_entries(df_meta_entire, dataloader):
    df_measurements = measurements_to_df(dataloader)
    print("df_measurements was created")
    indices_nan = df_meta_data[df_measurements["steer"].isna()].index.tolist()
    count_files_deleted = remove_entries_at_indices(indices_nan, df_meta_entire)
    return count_files_deleted

## Remove invalid entries of Noise Dataset

In [1]:
root_dir = os.path.join("..", "..", "data", "Noise-Dataset")
keep_input = ["lidar", "rgb", "measurements"] # "lidar"

In [2]:
def remove_unused_sensors_to_new_folder(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                shutil.rmtree(root)

In [ ]:
def remove_all_not_rgb_existent_files(root_dir):
        for (root, dirs, files) in os.walk(root_dir, topdown=True):
            # Current folder contains the files
            if not dirs:
                dir, input_type = os.path.split(root)
                if input_type not in keep_input:
                    shutil.rmtree(root)

In [5]:
remove_unused_sensors_to_new_folder(root_dir, keep_input)

In [7]:
create_metadata_df(root_dir, ["rgb", "lidar", "measurements"])

Varying number files among input types: ../../data/Noise-Dataset/Town03_Scenario9_route12_02_01_19_12_29/measurements
Varying number files among input types: ../../data/Noise-Dataset/Town04_Scenario7_route26_02_01_21_55_45/measurements
Varying number files among input types: ../../data/Noise-Dataset/Town04_Scenario7_route12_02_01_21_14_01/measurements
Varying number files among input types: ../../data/Noise-Dataset/Town04_Scenario7_route30_02_01_22_13_52/measurements
Varying number files among input types: ../../data/Noise-Dataset/Town02_Scenario8_route15_02_01_17_42_20/measurements
Varying number files among input types: ../../data/Noise-Dataset/Town04_Scenario7_route10_02_01_21_08_27/measurements
Varying number files among input types: ../../data/Noise-Dataset/Town03_Scenario9_route2_02_01_18_42_57/measurements
Varying number files among input types: ../../data/Noise-Dataset/Town03_Scenario9_route13_02_01_19_14_37/measurements
Varying number files among input types: ../../data/Noise-

KeyError: "['measurements'] not in index"